In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import shutil

# Step 1 - Scraping

## NASA Mars News

In [7]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [8]:
response = requests.get(url)

In [9]:
soup = bs(response.text, 'html.parser')

In [10]:
results = soup.find('div', class_ = 'slide')
print(results.prettify())

<div class="slide">
 <div class="image_and_description_container">
  <a href="/news/8426/nasa-garners-7-webby-award-nominations/">
   <div class="rollover_description">
    <div class="rollover_description_inner">
     Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.
    </div>
    <div class="overlay_arrow">
     <img alt="More" src="/assets/overlay-arrow.png"/>
    </div>
   </div>
   <img alt="NASA Garners 7 Webby Award Nominations" class="img-lazy" data-lazy="/system/news_items/list_view_images/8426_Webby2019-320x240.jpg" src="/assets/loading_320x240.png"/>
  </a>
 </div>
 <div class="content_title">
  <a href="/news/8426/nasa-garners-7-webby-award-nominations/">
   NASA Garners 7 Webby Award Nominations
  </a>
 </div>
</div>



In [11]:
content_title = soup.find_all('div', class_ = "content_title")
title = []

for ct in content_title:
    if (ct.a):
         if (ct.a.text):
                title.append(ct.text)


    title_text = title[0]
    title_text = title_text.replace("\n\n", "")
    
print(title_text)

NASA Garners 7 Webby Award Nominations


In [12]:
description = soup.find('div', class_ = "rollover_description_inner")
print(description.prettify())

<div class="rollover_description_inner">
 Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.
</div>



In [57]:
desc = []
for dc in description:
    if (dc):
        desc.append(dc)
        
    description_ = dc.replace("\n","")
print(description_)
        


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.


## JPL Mars Space Images - Featured Image

In [13]:

# Use splinter to navigate the site and find the image url 
# for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

/usr/local/bin/chromedriver


In [14]:
# Visit Mars Space Images through splinter module
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)

In [15]:
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[3]/a/div/div[2]/img"


In [16]:
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [20]:
html_img =browser.html
soup = bs(html_img, "html.parser")
img_url = soup.find('img', class_='fancybox-image')["src"]

In [21]:
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_image))
print(base_url)

https://www.jpl.nasa.gov/


In [22]:
featured_image_url = base_url + img_url
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23137_hires.jpg


## Mars Weather

In [29]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

/usr/local/bin/chromedriver


In [25]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")
mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

In [28]:
print(mars_weather)

InSight sol 132 (2019-04-10) low -97.7ºC (-143.8ºF) high -16.3ºC (2.7ºF)
winds from the SW at 4.3 m/s (9.5 mph) gusting to 12.2 m/s (27.3 mph)
pressure at 7.30 hPapic.twitter.com/lW713h6MMh


## Mars Facts

In [41]:
url_facts = "https://space-facts.com/mars/"

In [42]:
table = pd.read_html(url_facts)

In [43]:
table_df = pd.DataFrame(table[0])
table_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [47]:
table_df.columns=['Index', 'Values']

In [48]:
table_df = table_df.set_index('Index')

In [49]:
table_df

,Values
Index,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [50]:
table_df.to_csv()

'Index,Values\nEquatorial Diameter:,"6,792 km"\nPolar Diameter:,"6,752 km"\nMass:,6.42 x 10^23 kg (10.7% Earth)\nMoons:,2 (Phobos & Deimos)\nOrbit Distance:,"227,943,824 km (1.52 AU)"\nOrbit Period:,687 days (1.9 years)\nSurface Temperature:,-153 to 20 °C\nFirst Record:,2nd millennium BC\nRecorded By:,Egyptian astronomers\n'

## Mars Hemispheres

1. Visit the USGS Astrogeology site here to obtain high resolution 
   images for each of Mar's hemispheres.
    
2. You will need to click each of the links to the hemispheres
   in order to find the image url to the full resolution image.
    
3. Save both the image url string for the full resolution 
   hemisphere image, and the Hemisphere title containing 
   the hemisphere name. Use a Python dictionary to store 
   the data using the keys img_url and title.
    
4. Append the dictionary with the image url string and the 
   hemisphere title to a list. This list will contain 
   one dictionary for each hemisphere.

In [ ]:
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [240]:
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)

In [242]:
# for loop to get path for each imgs
# xpath1 = "//*[@id='product-section']/div[2]/div[i]/a/img"
xpath1 = []
hemi_url = []

# Get path for each hemisphere website
for i in range(1,5):
    x = f"//*[@id='product-section']/div[2]/div[{i}]/a/img"
    xpath1.append(x)
    
# Visit each image website from hemisphere website   
for x in xpath1:
    results = browser.find_by_xpath(x)
    img = results[0]
    img.click()

# Trying to obtain hemisphere urls
#     html = browser.html
#     soup = bs(html, 'html.parser')
#     img_url = soup.find("img")["src"]
#     hemi_url.append({"img_url": img_url})

    x = "//*[@id='wide-image']/div/ul/li/a"
    results = browser.find_by_xpath(xpath3)
    img = results[0]
    img.click()
    
# Use the requests library to download and save the image from the `img_url` above
    
    
    url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url_hemisphere)

MissingSchema: Invalid URL '/images/usgs_logo_main_2x.png': No schema supplied. Perhaps you meant http:///images/usgs_logo_main_2x.png?

In [239]:
xpath1

["//*[@id='product-section']/div[2]/div[1]/a/img",
 "//*[@id='product-section']/div[2]/div[2]/a/img",
 "//*[@id='product-section']/div[2]/div[3]/a/img",
 "//*[@id='product-section']/div[2]/div[4]/a/img"]

In [26]:
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[3]/a/div/div[2]/img"
results = browser.find_by_xpath(xpath)
img = results[0]
img.append()

In [236]:
# Obtaining each url
url_hemisphere2 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
browser.visit(url_hemisphere2)
hemi_url = []
html = browser.html
soup = bs(html, 'html.parser')

xp = "/[@class='container']/div[3]//h2.text" # How do I get path for title?
results = browser.find_by_xpath(xp)
hemi_url.append({"img_url": results})
#hemi_url.append({"title": title, "img_url": img_url})


#/html/body/div/span[@class='regular_price']


In [237]:
hemi_url

[{'img_url': []}]

In [ ]:
# Use the requests library to download and save the image from the `img_url` above
response = requests.get(img_url, stream=True)
with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)